In [1]:
import numpy as np
from pcm_construction.bbcodes import bbpcm
from src.noise.data_qubit import depolarizing_round
from src.closed_branch_decoder import CB_decoder

In [2]:
# We will first create the parity check matrix. For this example, it will be l = 6, m = 6 A_poly = 
l = 6
m = 6 
# A polynomial = x^3+y+y^2
A_poly = [[0,3], [1,1], [1,2]]
# B polynomial = y^3 + x + x^2
B_poly = [[1,3], [0,1], [0,2]]

H, Hx, Hz = bbpcm(l = l, m = m, A_poly= A_poly, B_poly = B_poly)

print(H.shape)
p = 0.03 #10%
error = depolarizing_round(p, H.shape[1])
print(error[:H.shape[1]//2])
print(error[H.shape[1]//2:])

n_branches = 10
n_growths = 5
model = (H, p)            

myDecoder = CB_decoder(model, max_branches = n_branches, max_growths = n_growths)

syndrome = np.dot(H, error) % 2
print(syndrome.shape)

error_recovered = myDecoder.decode(syndrome, comments = True)

(72, 144)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(72,)


In [10]:
l = 6
m = 6 
# A polynomial = x^3+y+y^2
A_poly = [[0,3], [1,1], [1,2]]
# B polynomial = y^3 + x + x^2
B_poly = [[1,3], [0,1], [0,2]]

S_l = np.roll(np.eye(l), 1, axis = 1)
S_m = np.roll(np.eye(m), 1, axis = 1)

x = np.kron(S_l, np.eye(m))
y = np.kron(np.eye(l), S_m)

A = np.zeros((l*m, l*m))

for element in A_poly:
    if element[0] == 0:
        A = (A + np.linalg.matrix_power(x, element[1])) % 2
    elif element[0] == 1:
        A = (A + np.linalg.matrix_power(y, element[1])) % 2
    print(np.where(A[:, 1] == 1)[0])
B = np.zeros((l*m, l*m))
for element in B_poly:
    if element[0] == 0:
        B = (B + np.linalg.matrix_power(x, element[1])) % 2
    elif element[0] == 1:
        B = (B + np.linalg.matrix_power(y, element[1])) % 2
    print(np.where(B[:, 1] == 1)[0])

Hx = np.hstack((A, B))
Hz = np.hstack((B.T, A.T))
zero_contribution = np.zeros(Hx.shape)
top = np.hstack((Hx, zero_contribution))
bottom = np.hstack((zero_contribution, Hz))
H = np.vstack((top, bottom))

[19]
[ 0 19]
[ 0  5 19]
[4]
[ 4 31]
[ 4 25 31]
